In [ ]:
import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error

from tqdm.notebook import tqdm
import os, gc
import random
from PIL import Image
import tifffile as tiff
import cv2
import zipfile
import collections
from PIL import Image
from sklearn import preprocessing
from random import randint
from glob import glob
import shutil

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

import torchvision
from torchvision import transforms
import albumentations as A
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import CosineAnnealingLR

import warnings
warnings.filterwarnings("ignore")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 2020
seed_everything(seed)
sz = 224
NFOLDS = 5

#ImageNet
mean = np.array([[[0.485, 0.456, 0.406]]])
std = np.array([[[0.229, 0.224, 0.225]]])

In [ ]:
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_ids = test_df.Id.to_list()
test_dir = "/kaggle/input/petfinder-pawpularity-score/test/"
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

In [ ]:
!python detect.py\
--weights /kaggle/input/ultralyticsyolov5aweights/yolov5x.pt\
--class 15 16\
--img 512\
--conf 0.3\
--iou 0.5\
--source $test_dir\
--name inference\
--save-txt --save-conf --exist-ok

In [ ]:
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
for n, image_id in tqdm(enumerate(test_ids)):
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    try:
        file_path = f'/kaggle/working/yolov5/runs/detect/inference/labels/{image_id}.txt'
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        for i, d in enumerate(data):
            xmin.append((d[2]-d[4]/2)*width)
            ymin.append((d[3]-d[5]/2)*height)
            xmax.append((d[2]+d[4]/2)*width)
            ymax.append((d[3]+d[5]/2)*height)
        all_xmin = int(min(xmin))
        all_ymin = int(min(ymin))
        all_xmax = int(max(xmax))
        all_ymax = int(max(ymax))
        all_width_half = (all_xmax - all_xmin) // 2
        all_height_half = (all_ymax - all_ymin) // 2
        r = np.maximum(all_width_half, all_height_half)
        all_xc = (all_xmin + all_xmax) // 2
        all_yc = (all_ymin + all_ymax) // 2
        final_xmin = np.maximum(all_xc-r, 0)
        final_ymin = np.maximum(all_yc-r, 0)
        final_xmax = np.minimum(all_xc+r, width)
        final_ymax = np.minimum(all_yc+r, height)
        crop_img = orig_image[final_ymin:final_ymax, final_xmin:final_xmax, :]
        crop_img = cv2.resize(crop_img, (sz, sz)).astype(np.uint8)
        np.save(save_dir + f'{image_id}', crop_img)
    except:
        orig_image = cv2.resize(orig_image, (sz, sz)).astype(np.uint8)
        np.save(save_dir + f'{image_id}', orig_image)

In [ ]:
!rm -r /kaggle/working/yolov5

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.loc[idx, 'Id']
        img = np.load(f'/kaggle/working/crop_images/{image_id}.npy').astype(np.float32)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img = cv2.resize(img, (sz, sz)).astype(np.float32)
        
        if self.transform:
            sample = self.transform(image=img)
            img = sample['image']
        
        img = (img/255.0 - mean) / std
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img)

        return img


def inference_fn(data_loader, model, device):
    model.eval()    
    val_preds = []
    
    for i, x in enumerate(data_loader):
        img = x
        img = img.to(device, dtype=torch.float)
        
        with torch.no_grad():
            pred = model(img)
            val_preds.append(nn.Softmax()(pred).detach().cpu().numpy())
            
    val_preds = np.concatenate(val_preds)
                
    return val_preds


class Model(nn.Module):
    def __init__(self, model_name='swin_large_patch4_window7_224_in22k', pretrained=False):
        super().__init__()
        self.model = timm.create_model(
            model_name=model_name, 
            in_chans=3, 
            pretrained=pretrained
            )
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, 100)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
test_ds = Dataset(df=test_df, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=32, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions = 0

for fold in range(NFOLDS):
    print(f'=============== FOLD:{fold} ===============')
    model = Model()
    model.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-010/swint1_fold_{fold}.pth'))
    model.to(device)
    prediction = inference_fn(test_dl, model, device)
    predictions += (target_cols.astype(np.int32).reshape(-1, 100) * prediction.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
test_df['Pawpularity'] = predictions

In [ ]:
!rm -r /kaggle/working/crop_images

In [ ]:
test_df

In [ ]:
test_df.to_csv('/kaggle/working/submission.csv', index=False)